Scenario 2 : There is a cross-functional team  team with one data scientist working on ML model.

MLFLOW setup:
tracking server : yes, local server,
backend store : sqlite database,
artifacte store : local file system.

To run this example we need to launch the mlflow server locally by running the following command in the terminal:
mlflow server --backend-stoe-uri sqlite:///backend.db

In [1]:
import mlflow

In [2]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [4]:
print(f"tracking URI : '{mlflow.get_tracking_uri()}'")

tracking URI : 'http://127.0.0.1:5000'


In [5]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1699188287542, experiment_id='0', last_update_time=1699188287542, lifecycle_stage='active', name='Default', tags={}>]

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score


mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():
    X,y = load_iris(return_X_y = True)

    params = {"C":0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X,y)
    y_pred = lr.predict(X)

    mlflow.log_metric("accuracy", accuracy_score(y,y_pred))

    mlflow.sklearn.log_model(lr,artifact_path = 'models')

    print(f"default artifacts URI : '{mlflow.get_artifact_uri()}'")



2023/11/05 18:20:25 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.


default artifacts URI : 'mlflow-artifacts:/1/b582972ba9ee4b749494d65d96190186/artifacts'


In [7]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1699188625662, experiment_id='1', last_update_time=1699188625662, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1699188287542, experiment_id='0', last_update_time=1699188287542, lifecycle_stage='active', name='Default', tags={}>]

Interacting with Model Registry

In [8]:
from mlflow.tracking import MlflowClient

client = MlflowClient("http://127.0.0.1:5000")

In [12]:
client.search_registered_models()

[]

In [32]:
#registring one model


#run_id = client.search_runs(experiment_id = '1')[0].run_id
run_id = client.search_runs(experiment_ids = '1')[0]
mlflow.register_model(
    model_uri = f"runs:/{run_id}/models",
    name = "iris-classifier"
)
#print(run_id)

Registered model 'iris-classifier' already exists. Creating a new version of this model...


RestException: RESOURCE_DOES_NOT_EXIST: Run with id=<Run: data=<RunData: metrics={'accuracy': 0.96}, params={'C': '0.1', 'random_state': '42'}, tags={'mlflow.log-model.history': '[{"run_id": "b582972ba9ee4b749494d65d96190186", '                             '"artifact_path": "models", "utc_time_created": '                             '"2023-11-05 12:50:26.398302", "flavors": '                             '{"python_function": {"model_path": "model.pkl", '                             '"predict_fn": "predict", "loader_module": '                             '"mlflow.sklearn", "python_version": "3.9.18", '                             '"env": {"conda": "conda.yaml", "virtualenv": '                             '"python_env.yaml"}}, "sklearn": '                             '{"pickled_model": "model.pkl", '                             '"sklearn_version": "1.3.1", '                             '"serialization_format": "cloudpickle", "code": '                             'null}}, "model_uuid": '  